In [15]:
#! pip install skl2onnx onnxruntime

In [28]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType

# Define intercept and coefficients
# from https://www.predictcancer.ai/Main.php?page=TFDInfo and https://doi.org/10.1016/j.clnu.2019.11.033 
intercept = -0.506
coefficients = np.array([-0.042, -0.030, 0.452, 0.608, -0.510, 0.311, 0.561, -0.655, 0.015, 0.042])  
# Coefficients for features:
# BMI - pretreatment weight change - modified diet or tube feeding (TF) [yes=1]) - WHO performance scale (PS) [WHO>0=1]) 
# tumor location [oral cavity, oropharynx and hypopharynx = 1]) - Tumor classification [T2, T3, or T4 = 1] 
# Node classification [N2 or N3 = 1] - systemic therapy [Cetuximab = 1] - 
# mean RT dose on contralateral submandibular gland [Gy] - mean RT dose on contralateral parotid salivary gland [Gy]


# Create logistic regression object
log_reg = LogisticRegression()

# Assign manually defined intercept and coefficients
log_reg.intercept_ = np.array([intercept])
log_reg.coef_ = np.array([coefficients])
log_reg.feature_names_in_ = ["BMI", "WeightLoss", "TF", "PS", "Tumorlocation", "Tclassification",
                            "Nclassification", "Systherapy", "RTdose_subman", "RTdosesalivary"]

def infer_initial_types():
  return [
      ('input', FloatTensorType([None, 10])) # you need to specify the amount of variables
  ]

log_reg.infer_initial_types = infer_initial_types

# Manually set the classes_
log_reg.classes_ = np.array([0, 1])  # binary task Example classes, replace with your actual classes if needed


# You may also need to set other parameters like solver and penalty
# log_reg.penalty = 'l2'
# log_reg.solver = 'liblinear'

# Now your logistic regression model is set with the provided intercept and coefficients
print(log_reg)
# print(log_reg.feature_names_in_)

LogisticRegression()


In [30]:
# print(log_reg.predict(np.array([1,0,1]).reshape(1, -1)))
prediction_dict = [{"BMI":19.5, "WeightLoss":-8, "TF":1, "PS":0, 
                                                       "Tumorlocation":1, "Tclassification":1,
                                                       "Nclassification":1, "Systherapy":0, 
                                                       "RTdose_subman":36, "RTdosesalivary":29}]
log_reg.predict_proba(pd.DataFrame(prediction_dict))

array([[0.18437244, 0.81562756]])

In [32]:
# Convert into ONNX format.
from skl2onnx import to_onnx

onx = to_onnx(log_reg, initial_types=log_reg.infer_initial_types())
with open("willemsen_tubefeed.onnx", "wb") as f:
    f.write(onx.SerializeToString())
    print('success')

success


In [34]:
## Read ONNX file, and perform prediction

import onnxruntime as ort
import numpy as np
ort_sess = ort.InferenceSession("willemsen_tubefeed.onnx")
outputs = ort_sess.run(None, {'input': [[19.5,-8,1,0,1,1,1,0,36,29]]})

# Print Result
outputs[1][0]

{0: 0.1843724548816681, 1: 0.8156275749206543}